# EDA

In [1]:
import pandas as pd
import numpy as np
import sweetviz
from sklearn.model_selection import train_test_split

# titanic data

In [2]:
path = '../../data/titanic.csv'
df = pd.read_csv(path)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# X, y split
X = df.drop('Survived', axis=1)
y = df['Survived']

In [4]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Flaml

In [5]:
from flaml import AutoML
automl = AutoML()

In [6]:
# 모델 튜닝하기
settings = {
    "time_budget": 60,  # total running time in seconds
    "metric": 'rmse',  # can be: 'r2', 'rmse', 'mae', 'mse', 'accuracy', 'roc_auc', 'roc_auc_ovr',
                           # 'roc_auc_ovo', 'log_loss', 'mape', 'f1', 'ap', 'ndcg', 'micro_f1', 'macro_f1'
    "task": 'classification',  # task type    
    "log_file_name": 'titanic.log',  # flaml log file
    "seed": 421266,    # random seed
}

# 파라미터 참고 링크
# https://microsoft.github.io/FLAML/

In [7]:
automl.fit(X_train=X_train, y_train=y_train, **settings)

[flaml.automl: 11-11 23:11:29] {1485} INFO - Data split method: stratified
[flaml.automl: 11-11 23:11:29] {1489} INFO - Evaluation method: cv
[flaml.automl: 11-11 23:11:29] {1540} INFO - Minimizing error metric: rmse
[flaml.automl: 11-11 23:11:29] {1577} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'lrl1']
[flaml.automl: 11-11 23:11:29] {1826} INFO - iteration 0, current learner lgbm
[flaml.automl: 11-11 23:11:30] {1943} INFO - Estimated sufficient time budget=1410s. Estimated necessary time budget=23s.
[flaml.automl: 11-11 23:11:30] {2023} INFO -  at 0.2s,	estimator lgbm's best error=0.4557,	best estimator lgbm's best error=0.4557
[flaml.automl: 11-11 23:11:30] {1826} INFO - iteration 1, current learner lgbm
[flaml.automl: 11-11 23:11:30] {2023} INFO -  at 0.2s,	estimator lgbm's best error=0.4557,	best estimator lgbm's best error=0.4557
[flaml.automl: 11-11 23:11:30] {1826} INFO - iteration 2, current learner lgbm
[flaml.automl: 11-11 23:11:30] {20

# Flaml 결과

In [8]:
''' retrieve best config and best learner'''
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 7, 'num_leaves': 5, 'min_child_samples': 21, 'learning_rate': 0.32556254798353695, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.4786212688867904}
Best accuracy on validation data: 0.5974
Training duration of best run: 0.00672 s


In [9]:
automl.model.estimator

LGBMClassifier(learning_rate=0.32556254798353695, max_bin=255,
               min_child_samples=21, n_estimators=7, num_leaves=5,
               reg_alpha=0.0009765625, reg_lambda=1.4786212688867904,
               verbose=-1)

In [11]:
''' compute predictions of testing dataset ''' 
y_pred = automl.predict(X_test)
print('Predicted labels', y_pred)
print('True labels', y_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]


Predicted labels [0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1
 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1]
True labels 709    1
439    0
840    0
720    1
39     1
      ..
433    0
773    0
25     1
84     1
10     1
Name: Survived, Length: 179, dtype: int64


In [12]:
''' compute different metric values on testing dataset'''
from flaml.ml import sklearn_metric_loss_score
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred, y_test))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test))


accuracy = 0.8100558659217877
roc_auc = 0.8755469755469756
log_loss = 0.44168372830689157
